In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import warnings
import os

from omegaconf import OmegaConf
import pandas as pd
from sklearn.neighbors import KDTree
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.table import Table
from astropy.io.fits.verify import VerifyWarning

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

In [ ]:
from astroquery.esa.euclid.core import Euclid
Euclid.login()

In [ ]:
cfg_dict = dict(
    # TODO change this path
    base_dir='/media/home/team_workspaces/Galaxy-Zoo-Euclid/data/pipeline_runs',
    name='external_targets_debug',
    release_name='F-006',
    bands=['VIS', 'NIR_Y']
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc

In [ ]:
external_targets = pd.read_csv('/media/user/repos/bulk-euclid-cutouts/tests/data/102021539_mer_catalog.csv')

In [ ]:
targets_with_tiles = pipeline.get_matching_tiles(cfg)

In [ ]:
pipeline.make_cutouts(cfg, tiles, target_tiles)